In [3]:
import os
import pydicom
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import shutil
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.optimizers import Adadelta
from keras.backend import get_session
from keras.preprocessing.image import ImageDataGenerator

In [4]:
# For left direction skull DICOM
leftdir_path = 'E:/segmentationstudy/image/leftdir'
leftdir_list = []
for (path, dir, files) in os.walk(leftdir_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            leftdir_list.append(filePath)

In [7]:
# For right direction skull DICOM
rightdir_path = 'E:/segmentationstudy/image/rightdir'
rightdir_list = []
for (path, dir, files) in os.walk(rightdir_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            rightdir_list.append(filePath)

In [8]:
print(len(leftdir_list))
print(len(rightdir_list))

101
45


In [10]:
### Create dataframe
# leftdir_list = 0
# rightdir_list = 1

leftdir_df = pd.DataFrame(data={'filename': leftdir_list, 'label': [0 for i in range(len(leftdir_list))]})
rightdir_df = pd.DataFrame(data={'filename': rightdir_list, 'label': [1 for i in range(len(rightdir_list))]})

df=leftdir_df.append(rightdir_df)
df.columns=['filename', 'label']
df=df.sample(frac=1)
df.reset_index(drop=True,inplace=True)

In [13]:
### One hot encoding of label
y_train = np.array(df['label'].tolist())
y_train = keras.utils.np_utils.to_categorical(y_train)

In [16]:
x_train=[]
for i in df['filename']:
    pixeldata = pydicom.dcmread(i).pixel_array
    pixeldata = cv2.resize(pixeldata, (128, 128))
    mean, std = pixeldata.mean(), pixeldata.std()
    x_train.append((np.clip((pixeldata-mean)/std, -2.0, 2.0)+2.0)/4.0)
x_train=np.array(x_train).reshape(146, 128, 128, 1)

In [20]:
x_val=x_train[100:126]
y_val=y_train[100:126]
x_test=x_train[126:]
y_test=y_train[126:]

In [21]:
x_train=x_train[:100]
y_train=y_train[:100]

In [23]:
#DCNN
def cnn(size):
    KERNEL = (8,8)
    shape = (size[0], size[1], size[2])
    
    # Define Model
    model = Sequential()
    
    # Add Convolution 1
    model.add(Conv2D(filters=8, kernel_size=KERNEL, activation='relu', input_shape=shape, padding='same'))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 2
    model.add(Conv2D(filters=16, kernel_size=KERNEL, activation='relu', input_shape=shape, padding='same'))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 3
    model.add(Conv2D(filters=32, kernel_size=KERNEL, activation='relu', input_shape=shape, padding='same'))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    
    # Add Output Layer
    model.add(Dropout(0.5))
    model.add(Dense(units=2, activation='softmax'))
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    model.summary()
    
    return model

model = cnn(size=x_train.shape[1:])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 8)       520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)       

In [26]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=4,  validation_data=(x_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 100 samples, validate on 26 samples
Epoch 1/50
100/100 [==============================] - 3s 26ms/step - loss: 0.9133 - accuracy: 0.6800 - val_loss: 0.5940 - val_accuracy: 0.6923
Epoch 2/50
100/100 [==============================] - 2s 21ms/step - loss: 0.5586 - accuracy: 0.7200 - val_loss: 0.5604 - val_accuracy: 0.6923
Epoch 3/50
100/100 [==============================] - 2s 20ms/step - loss: 0.5341 - accuracy: 0.7500 - val_loss: 0.5362 - val_accuracy: 1.0000
Epoch 4/50
100/100 [==============================] - 2s 19ms/step - loss: 0.4312 - accuracy: 0.8400 - val_loss: 0.3314 - val_accuracy: 1.0000
Epoch 5/50
100/100 [==============================] - 2s 19ms/step - loss: 0.2679 - accuracy: 0.8300 - val_loss: 0.2336 - val_accuracy: 1.0000
Epoch 6/50
100/100 [==============================] - 2s 20ms/step - loss: 0.2117 - accuracy: 0.9100 - val_loss: 0.1315 - val_accuracy: 1.0000
Epo

In [29]:
# 테스트 데이터로 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

20/20 [==============================] - 0s 3ms/step
loss= 2.559922904765699e-05
accuracy= 1.0


In [30]:
os.getcwd()

'C:\\Users\\ABMI\\Desktop\\git\\One_Commit_per_Day\\SkullFracture'

In [31]:
# 모델 저장하기
from keras.models import load_model
model.save('C:/Users/ABMI/Desktop/git/One_Commit_per_Day/SkullFracture/skull_leftright/skull_leftright_model.h5')

In [ ]:
# 2. 모델 불러오기
from keras.models import load_model
model = load_model('mnist_mlp_model.h5')

# 3. 모델 사용하기
yhat = model.predict_classes(xhat)